In [3]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
# from node import node
from copy import deepcopy

# TDI

In [ ]:
# save TDI information
A_Di = pd.read_csv("chunhui/Input/HNSC/DriverDEGmatrix.HNSC.csv",
                   index_col=0,
                   header=0)
d = {}
for i in A_Di.index:
    deg_l = [j for j in A_Di.columns if A_Di.loc[i, j] == 1]
    d.update({i: deg_l})

import json
with open('data.json', 'w') as outfile:
    json.dump(d, outfile, indent=4)

S_Mi = pd.read_csv("chunhui/Input/HNSC/S_Mi.csv", index_col=0, header=0)
S_Mi.index = [ele[0:12] for ele in S_Mi.index]

# Mutation and CNV

In [5]:
# Mutation transform
S_A = pd.read_csv("../TCI/EM/Input/S_A.csv", index_col=0, header=0)
S_Ai = pd.read_csv("chunhui/Input/HNSC/SGAmatrix.HNSC.csv",
                   index_col=0,
                   header=0)
A_Di = pd.read_csv("chunhui/Input/HNSC/DriverDEGmatrix.HNSC.csv",
                   index_col=0,
                   header=0)
S_M = pd.read_csv("chunhui/Input/HNSC/TCGA/HNSC_mc3_gene_level.txt",
                  sep='\t',
                  index_col=0).T
S_C = pd.read_csv("chunhui/Input/HNSC/TCGA/TCGA-HNSC.gistic.tsv",
                  sep='\t',
                  index_col=0).T


In [ ]:
# transfer gene symble
import mygene

mg = mygene.MyGeneInfo()
ens = [ele[0:15] for ele in S_C.columns]
S_C.columns = ens
S_C.to_csv("chunhui/Input/HNSC/TCGA/cnv.csv", index=True, header=True)
geneinfo = mg.querymany(ens, scopes='ensembl.gene')

em_l = []
deg_l = []
for ele in geneinfo:
    if ele["query"] not in em_l:
        em_l.append(ele["query"])
        if "notfound" not in ele.keys():
            deg_l.append(ele['symbol'])
        else:
            deg_l.append(ele['query'])

S_C.columns = deg_l

In [9]:
S_C.to_csv("chunhui/Input/HNSC/TCGA/cnv_symbol.csv", index=True, header=True)

# Expression

In [ ]:
# read expression data
S_Dcai = pd.read_csv("chunhui/Input/HNSC/TCGA-HNSC.htseq_fpkm-uq.tsv",
                     sep='\t',
                     index_col=0).T

# transfer gene symble
import mygene

mg = mygene.MyGeneInfo()

ens = [ele[0:15] for ele in S_Dcai.columns]
geneinfo = mg.querymany(ens, scopes='ensembl.gene')

em_l = []
deg_l = []
for ele in geneinfo:
    if ele["query"] not in em_l:
        em_l.append(ele["query"])
        if "notfound" not in ele.keys():
            deg_l.append(ele['symbol'])
        else:
            deg_l.append(ele['query'])
S_Dcai.columns = deg_l
S_Dcai.to_csv("chunhui/Output/HNSC/S_Dcai.csv", header=True, index=True)

In [ ]:
# PAM 50 subtype
dfs = sur_dfs

fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

km = KaplanMeierFitter()

for i in range(1, 6):
    df_m = dfs.loc[set(BRCA_sub.index[BRCA_sub["number"] == i])
                   & set(dfs.index)]
    i = str(i)
    locals()["T_" + i] = df_m['%s.time' % par]  # /365
    locals()["E_" + i] = df_m[par]
    km.fit(locals()["T_" + i],
           event_observed=locals()["E_" + i],
           label=sb_l[int(i)])
    km.survival_function_.plot(ax=ax1)
    km.plot(ax=ax2)

ax1.set_title("PRO Subgroup")
ax1.set_xlabel(None)
ax1.set_xlim(0, 2000)

for i in range(1, 6):
    for j in range(1, 6):
        if j > i:
            lr = logrank_test(locals()["T_%s" % i],
                              locals()["T_%s" % j],
                              locals()["E_%s" % i],
                              locals()["E_%s" % j],
                              alpha=.99)
            print("p_value of %s and %s: " % (sb_l[int(i)], sb_l[int(j)]) +
                  str(lr.p_value))
